# Chapter 11 - Deep Network on CIFAR10 Dataset

In [57]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd 
import numpy as np 
import ssl
import matplotlib.pyplot as plt 
from functools import partial 
import os

In [58]:
ssl._create_default_https_context = ssl._create_unverified_context

In [59]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

In [60]:
X_train, y_train, X_valid, y_valid = X_train_full[:40000], y_train_full[:40000], X_train_full[40000:], y_train_full[40000:]

Default DNN Architecture 
- LeCun Initialiation
- ELU Activation

In [61]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

DenseLayer = partial(
    keras.layers.Dense, 
    activation = 'elu',
    kernel_initializer = 'he_normal'
)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(DenseLayer(100))

In [62]:
model.add(keras.layers.Dense(10, activation='softmax'))

In [63]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 100)               307300    
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 100)               1

In [64]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('models/ch11_model.h5', save_best_only=True)
run_index = 1
run_logdir = os.path.join(os.curdir, 'cifar10_logs', 'run_{:03d}'.format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [65]:
# default learning rate works alright, but not as good as 5e-5
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Nadam(learning_rate=5e-5),
              metrics=["accuracy"])

In [66]:
model.fit(
    X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[callbacks]
)

Epoch 1/100
1250/1250 [==============================] - 18s 12ms/step - loss: 4.7200 - accuracy: 0.1420 - val_loss: 2.2326 - val_accuracy: 0.1847
Epoch 2/100
1250/1250 [==============================] - 16s 13ms/step - loss: 2.1254 - accuracy: 0.2159 - val_loss: 2.0535 - val_accuracy: 0.2408
Epoch 3/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.9986 - accuracy: 0.2634 - val_loss: 1.9302 - val_accuracy: 0.2881
Epoch 4/100
1250/1250 [==============================] - 16s 13ms/step - loss: 1.9035 - accuracy: 0.3004 - val_loss: 1.8619 - val_accuracy: 0.3117
Epoch 5/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.8382 - accuracy: 0.3273 - val_loss: 1.8116 - val_accuracy: 0.3422
Epoch 6/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.7964 - accuracy: 0.3431 - val_loss: 1.8478 - val_accuracy: 0.3218
Epoch 7/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.7539 - accuracy: 0.3616 - val_loss: 1

In [67]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.5956 - accuracy: 0.4578


[1.5956181287765503, 0.4578000009059906]

Using Batch Normalization 

In [68]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [69]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer='he_normal'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('elu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [70]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 batch_normalization (BatchN  (None, 3072)             12288     
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 100)               307300    
                                                                 
 batch_normalization_1 (Batc  (None, 100)              400       
 hNormalization)                                                 
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               1

In [71]:
# callback method to decrease lr exponentially after 10 epochs
# NOT FOR FINDING OPTIMAL RATE, METHOD OF TRAINING THAT DECREASES LR AS CONVERGENCE BEGINS
def exp_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    return lr * tf.math.exp(-0.1)

In [72]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(exp_scheduler)

In [73]:
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('models/ch11_model_batchnorm.h5', save_best_only=True)
run_logdir = os.path.join(os.curdir, 'cifar10_logs_batchnorm', 'run_{:03d}'.format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [74]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(
    loss='sparse_categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'] 
)

In [75]:
history = model.fit(
    X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), 
    callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]
)

Epoch 1/100
1250/1250 [==============================] - 35s 20ms/step - loss: 1.8483 - accuracy: 0.3365 - val_loss: 1.6725 - val_accuracy: 0.3963
Epoch 2/100
1250/1250 [==============================] - 24s 19ms/step - loss: 1.6730 - accuracy: 0.4025 - val_loss: 1.6088 - val_accuracy: 0.4309
Epoch 3/100
1250/1250 [==============================] - 25s 20ms/step - loss: 1.6026 - accuracy: 0.4291 - val_loss: 1.5504 - val_accuracy: 0.4446
Epoch 4/100
1250/1250 [==============================] - 24s 19ms/step - loss: 1.5550 - accuracy: 0.4439 - val_loss: 1.4790 - val_accuracy: 0.4730
Epoch 5/100
1250/1250 [==============================] - 23s 18ms/step - loss: 1.5083 - accuracy: 0.4619 - val_loss: 1.4633 - val_accuracy: 0.4716
Epoch 6/100
1250/1250 [==============================] - 23s 19ms/step - loss: 1.4756 - accuracy: 0.4762 - val_loss: 1.4465 - val_accuracy: 0.4893
Epoch 7/100
1250/1250 [==============================] - 23s 18ms/step - loss: 1.4425 - accuracy: 0.4877 - val_loss: 1

In [76]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 1.4547 - accuracy: 0.5254


[1.4547209739685059, 0.5253999829292297]

Batch Norm and Convolutions 

In [84]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [85]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(32,32,3)))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer='he_normal'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('elu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [86]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 64)         0

In [87]:
# same learning rate with conv layers?
optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(
    loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']
)

In [88]:
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('models/ch11_model_batchnorm_conv.h5', save_best_only=True)
run_logdir = os.path.join(os.curdir, 'cifar10_logs_batchnorm_conv', 'run_{:03d}'.format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [89]:
conv_hisotry = model.fit(
    X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
    callbacks=[early_stopping_cb, model_checkpoint_cb, tensorboard_cb]
)

Epoch 1/100
1250/1250 [==============================] - 43s 28ms/step - loss: 1.7685 - accuracy: 0.3529 - val_loss: 1.7412 - val_accuracy: 0.3642
Epoch 2/100
1250/1250 [==============================] - 35s 28ms/step - loss: 1.4785 - accuracy: 0.4668 - val_loss: 1.4275 - val_accuracy: 0.4948
Epoch 3/100
1250/1250 [==============================] - 34s 27ms/step - loss: 1.3685 - accuracy: 0.5109 - val_loss: 1.7144 - val_accuracy: 0.4442
Epoch 4/100
1250/1250 [==============================] - 35s 28ms/step - loss: 1.2860 - accuracy: 0.5414 - val_loss: 1.3042 - val_accuracy: 0.5419
Epoch 5/100
1250/1250 [==============================] - 35s 28ms/step - loss: 1.2132 - accuracy: 0.5699 - val_loss: 1.1431 - val_accuracy: 0.5977
Epoch 6/100
1250/1250 [==============================] - 34s 28ms/step - loss: 1.1508 - accuracy: 0.5929 - val_loss: 1.6163 - val_accuracy: 0.4944
Epoch 7/100
1250/1250 [==============================] - 35s 28ms/step - loss: 1.1079 - accuracy: 0.6090 - val_loss: 1

In [ ]:
model.evaluate(X_test, y_test)

Add dropout to model